In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter # a helper library to map word->freq mapping
import re # regex library for string manipulation

### Text source kaggle: https://www.kaggle.com/datasets/jayashree4/fiction

In [2]:
def read_file(filename):
  with open(filename,'r') as file:
    file_content=file.read()
  file_content=file_content.lower() # turning all letters into lower case
  words_list=re.findall(r'\w+',file_content) # easiest way of Tokenizing the words
  word_freq_dict={}
  vocabulary=set(words_list)
  word_freq_dict=Counter(words_list)
  return list(vocabulary),word_freq_dict

In [3]:
vocab,freq_dict=read_file('/content/data.txt')

In [4]:
len(vocab)

15608

In [5]:
len(vocab),len(freq_dict)

(15608, 15608)

In [6]:
count=0
for key,value in freq_dict.items():
  print(key,value)
  if count==5:
    break
  count+=1

chapter 64
i 5913
in 4458
which 1290
phileas 238
fogg 627


### For auto-correction I should also have a probability of each word in the corpus so lets calculate it

In [7]:
def cal_prob(freq_dict_):
  '''
  we can calculate the probabilty of each word in the corpus
  by freq_of_word/length_of_words_in_vocabulary
  '''
  prob={}
  for key in freq_dict_.keys():
    prob[key]=freq_dict_[key]/sum(freq_dict_.values())
  return prob

In [8]:
prob=cal_prob(freq_dict)

In [9]:
def get_prob(word,prob=prob): # this will return the prob of the given word
  return prob.get(word,0)

In [10]:
word_entered='prwtty'

# Edit Distance
## We will use edit distance approach for auto-correction
### We know that edit distance have 3 major operations named: insert,delete,replace, so lets implement them


In [11]:
'''
we will implement the edit distance later in this notebook
but for now we will use english alphabet corpus to insert at every position in the word
, similarly we will delete each character and replace each character with some english
alphabet and at the end we will see the words that matches with our vocabulary and will check
the probability for them, I know it sounds quite weird but believe me its not , so lets dive into it !!!
'''
english_alpha='abcdefghijklmnopqrstuvwxyz'
def insert_word(word):
  split_words=[(word[:i],word[i:]) for i in range(len(word)+1)]
  insert_words=[ a+l+b for a,b in split_words for l in english_alpha]
  return insert_words



In [12]:
def replace_word(word):
  split_words=[(word[:i],word[i:]) for i in range(len(word)+1)]
  replace_words=[a+l+(b[1:] if len(b)> 1 else '') for a , b in split_words for l in english_alpha]
  return replace_words

In [13]:
def delete_word(word):
  split_words=[(word[:i],word[i:]) for i in range(len(word)+1)]
  delete_words=[a+b[1:] for a ,b in split_words]
  return delete_words

In [14]:
def edit_one(word):
  edit_letter_set=set()
  edit_letter_set.update(insert_word(word))
  edit_letter_set.update(replace_word(word))
  edit_letter_set.update(delete_word(word))
  return edit_letter_set

In [15]:
def edit_letters(word, letters=1):
    edit_letter_set = edit_one(word)
    for i in range(1, letters):
        new_edits = set()
        for word in edit_letter_set:
            nth_edit = edit_one(word)
            new_edits.update(nth_edit)
        edit_letter_set.update(new_edits)

    return edit_letter_set

In [16]:
def get_suggestions(word,vocab):
  '''
  We will perform 3 steps here
  first we will see the word in vocabulary if its exit then our work is done and we will return that word and words similar to that
  secondly if first condition doesnt hold then we will edit one letter and check from those words if not then we can also generate 2 letter or
  even high letter edits and can see for a matching word, Remember we are not looking for context , we are here just for correction
  '''
  sugg=list()
  if word in vocab:
    sugg.append(word)
  else:
    sugg.append(list(edit_letters(word,letters=2).intersection(vocab)))
  return sugg

In [17]:
'''
So lets store the probabilties of the new words
'''
words_sugg=get_suggestions(word_entered,vocab)
words_sugg=[item for sublist in words_sugg  for item in sublist]
prob_sugg=[(word,get_prob(word)) for word in words_sugg]

## Now we have words suggestions , now the next step will be to find the minimum edit distance

In [18]:
def min_edit_distance(source, target, ins_cost=1, del_cost=1, rep_cost=2):
    m = len(source)
    n = len(target)
    D = np.zeros((m+1, n+1), dtype=int)

    # Initializing the first row and column of the matrix with deletion and insertion costs
    for row in range(1, m+1):
        D[row, 0] = D[row-1, 0] + del_cost

    for col in range(1, n+1):
        D[0, col] = D[0, col-1] + ins_cost

    for row in range(1, m+1):
        for col in range(1, n+1):
            # Calculate the cost of replacement (r_cost)
            r_cost = rep_cost
            if source[row-1] == target[col-1]:
                r_cost = 0

            # Calculate the minimum edit distance at the current position
            D[row, col] = min([
                D[row-1, col] + del_cost,    # Deletion
                D[row, col-1] + ins_cost,    # Insertion
                D[row-1, col-1] + r_cost     # Replacement or no operation if characters are the same
            ])

    med = D[m, n] # min distance at last row and last col , in short last element in last row and last col
    return  med #I am not returning the matrix , if you want , you can adjust the code

In [19]:
source=word_entered
distances=[(word,min_edit_distance(source,word)) for word in words_sugg]

In [20]:
distances[:5]

[('pretty', 2), ('petty', 3)]

In [21]:
def choose_word(word_list):
    """
    Chooses the word with the minimum edit distance from the list of tuples.

    Parameters:
        word_list (list): List of tuples containing words and their edit distances.

    Returns:
        str: The word with the minimum edit distance.
    """
    min_distance = float('inf')  # Initialize with a large value
    chosen_word = None

    for word, distance in word_list:
        if distance < min_distance:
            min_distance = distance
            chosen_word = word

    return chosen_word

In [22]:
choose_word(distances)

'pretty'

## Lets implement all at once

In [23]:
word_entered='powr'
words_sugg=get_suggestions(word_entered,vocab)
words_sugg=[item for sublist in words_sugg  for item in sublist]
prob_sugg=[(word,get_prob(word)) for word in words_sugg]
source=word_entered
distances=[(word,min_edit_distance(source,word)) for word in words_sugg]
distances[:5] # showing only 5 elements

[('pair', 4), ('or', 2), ('door', 4), ('peer', 4), ('pours', 3)]

In [24]:
choose_word(distances)

'power'